## 1. Collect Evidence

In the second phase of SDMT, we collect _evidence_ to attest to the fact that the model realized the properties specified in the previous phase.

We define and instantiate `Measurement`s to generate this evidence. Each individual piece of evidence is a `Value`. Once `Value`s are produced, we can persist them to an _artifact store_ to maintain our evidence across sessions. 

#### Preliminaries

In [ ]:
# Preliminaries for loading the package locally
import os
import sys

def package_root() -> str:
    """Resolve the path to the project root."""
    return os.path.abspath(os.path.join(os.getcwd(), "..", "src/"))

sys.path.append(package_root())
sys.path.append(os.getcwd())

#### Initialize MLTE Context

MLTE contains a global context that manages the currently active _session_. Initializing the context tells MLTE how to store all of the artifacts that it produces.

In [ ]:
import mlte

store_path = os.path.join(os.getcwd(), "store")
os.makedirs(store_path, exist_ok=True)

mlte.set_model("OxfordFlower", "0.0.1")
mlte.set_artifact_store_uri(f"local://{store_path}")

#### Fairnesss Measurements

Evidence collected in this section checks for fairness.

In [ ]:
# General functions.

import garden
import numpy as np
from pathlib import Path

# The path at which datasets are stored
DATASETS_DIR = Path(os.getcwd()) / "data"


def load_data(data_folder: str):
    """Loads all garden data results and taxonomy categories."""
    df_results = garden.load_base_results(data_folder)
    df_results.head()

    # Load the taxonomic data and merge with results.
    df_info = garden.load_taxonomy(data_folder)
    df_results.rename(columns = {'label':'Label'}, inplace = True)
    df_all = garden.merge_taxonomy_with_results(df_results, df_info)

    return df_info, df_all


def split_data(df_info, df_all):
    """Splits the data into 3 different populations to evaluate them."""
    df_gardenpop = df_info.copy()
    df_gardenpop['Population1'] = (np.around(np.random.dirichlet
                            (np.ones(df_gardenpop.shape[0]),size=1)[0],
                            decimals = 3) *1000).astype(int)
    df_gardenpop['Population2'] = (np.around(np.random.dirichlet
                            (np.ones(df_gardenpop.shape[0]),size=1)[0],
                            decimals = 3) *1000).astype(int)
    df_gardenpop['Population3'] = (np.around(np.random.dirichlet
                            (np.ones(df_gardenpop.shape[0]),size=1)[0],
                            decimals = 3) *1000).astype(int)
    df_gardenpop

    #build populations from test data set that match the garden compositions
    from random import choices

    #build 3 gardens with populations of 1000.
    pop_names = ['Population1', 'Population2', 'Population3']
    gardenpops = np.zeros( (3,1000), int)
    gardenmems = np.zeros( (3,1000), int)

    for j in range(1000):
        for i in range(len(df_gardenpop)):
            my_flower = df_gardenpop.iloc[i]['Common Name']
        
            for g in range(3):
                n_choices = df_gardenpop.iloc[i][pop_names[g]]
                my_choices = df_all[df_all['Common Name'] == my_flower]['model correct'].to_list()
                my_selection = choices(my_choices, k=n_choices)
            
                gardenpops[g][j] += sum(my_selection)
                gardenmems[g][j] += len(my_selection)

    gardenpops

    return gardenpops, gardenmems


def calculate_model_performance_acc(gardenpops, gardenmems):
    """Get accucray of models across the garden populations"""
    gardenacc = np.zeros( (3,1000), float)
    for i in range (1000):
        for g in range(3):
            gardenacc[g][i] = gardenpops[g][i]/gardenmems[g][i]
    gardenacc

    model_performance_acc = []
    for g in range(3):
        avg = round(np.average(gardenacc[g][:]),3)
        std = round(np.std(gardenacc[g][:]),3)
        min = round(np.amin(gardenacc[g][:]),3)
        max = round(np.amax(gardenacc[g][:]),3)
        model_performance_acc.append(round(avg,3))
        
        print("%1d %1.3f %1.3f %1.3f %1.3f" % (g, avg, std, min, max))

    return model_performance_acc


In [ ]:

# Prepare the data.
data = load_data(DATASETS_DIR)
split_data = split_data(data[0], data[1])

In this first example, we simply wrap the output from `accuracy_score` with a custom `Result` type to cope with the output of a third-party library that is not supported by a MLTE builtin.

In [ ]:
from multiple_accuracy import MultipleAccuracy
from mlte.measurement import ExternalMeasurement

# Evaluate accuracy, identifier has to be the same one defined in the Spec.
accuracy_measurement = ExternalMeasurement("accuracy across gardens", MultipleAccuracy, calculate_model_performance_acc)
accuracy = accuracy_measurement.evaluate(split_data[0], split_data[1])

# Inspect value
print(accuracy)

# Save to artifact store
accuracy.save()

#### Robustness Measurements

Evidence collected in this section checks for robustness.

In [ ]:
# General functions.
import pandas as pd


def calculate_base_accuracy(df_results: pd.DataFrame) -> pd.DataFrame:
    #Calculate the base model accuracy result per data label
    df_pos = df_results[df_results['model correct'] == True].groupby('label').count()
    df_pos.drop(columns = ['prediced_label'], inplace = True)
    df_neg = df_results[df_results['model correct'] == False].groupby('label').count()
    df_neg.drop(columns = ['prediced_label'], inplace = True)
    df_neg.rename(columns = {'model correct':'model incorrect'}, inplace = True)
    df_res = df_pos.merge(df_neg, right_on ='label', left_on = 'label', how = 'outer')
    df_res.fillna(0, inplace = True)
    df_res['model acc'] = df_res['model correct'] / ( df_res['model correct'] + df_res['model incorrect'] )
    df_res['count'] = ( df_res['model correct'] + df_res['model incorrect'] )
    df_res.drop(columns = ['model correct', 'model incorrect'], inplace = True)
    df_res.head()

    return df_res

def calculate_accuracy_per_set(data_folder: str, df_results: pd.DataFrame, df_res: pd.DataFrame) -> pd.DataFrame:
    #Calculate the model accuracy per data label for each blurred data set
    base_filename = 'FlowerModelv1_TestSetResults'
    ext_filename = '.csv'
    set_filename = ['_blur2x8', '_blur5x8', '_blur0x8', '_noR', '_noG','_noB']

    col_root = 'model acc'

    for fs in set_filename:
        filename = os.path.join(data_folder, base_filename + fs + ext_filename)
        colname = col_root + fs
        
        df_temp = pd.read_csv(filename)
        df_temp.drop(columns = ['Unnamed: 0'], inplace = True)
        
        df_pos = df_temp[df_temp['model correct'] == True].groupby('label').count()
        df_pos.drop(columns = ['prediced_label'], inplace = True)
        df_neg = df_results[df_results['model correct'] == False].groupby('label').count()
        df_neg.drop(columns = ['prediced_label'], inplace = True)
        df_neg.rename(columns = {'model correct':'model incorrect'}, inplace = True)
        df_res2 = df_pos.merge(df_neg, right_on ='label', left_on = 'label', how = 'outer')
        df_res2.fillna(0, inplace = True)
        
        df_res2[colname] = df_res2['model correct'] / ( df_res2['model correct'] + df_res2['model incorrect'] )
        df_res2.drop(columns = ['model correct', 'model incorrect'], inplace = True)
        
        df_res = df_res.merge(df_res2, right_on = 'label', left_on = 'label', how = 'outer')

    df_res.head()
    return df_res

In [ ]:
# Prepare all data.
df_results = garden.load_base_results(DATASETS_DIR)
df_res = calculate_base_accuracy(df_results)
df_res = calculate_accuracy_per_set(DATASETS_DIR, df_results, df_res)
df_info = garden.load_taxonomy(DATASETS_DIR)
df_all = garden.merge_taxonomy_with_results(df_res, df_info, "label", "Label")

#fill in missing model accuracy data
df_all['model acc_noR'].fillna(0, inplace = True)
df_all['model acc_noG'].fillna(0, inplace = True)
df_all['model acc_noB'].fillna(0, inplace = True)

Now do the actual measurements.

In [ ]:
#view changes in model accuracy
model_acc = sum( df_res['model acc'] * df_res['count'] ) / sum( df_res['count'] )
print('base model accuracy' , model_acc)
model_acc = sum( df_res['model acc_blur2x8'] * df_res['count'] ) / sum( df_res['count'] )
print('model accuracy with 2x8 blur' , model_acc)
model_acc = sum( df_res['model acc_blur5x8'] * df_res['count'] ) / sum( df_res['count'] )
print('model accuracy with 5x8 blur' , model_acc)
model_acc = sum( df_res['model acc_blur0x8'] * df_res['count'] ) / sum( df_res['count'] )
print('model accuracy with 0x8 blur' , model_acc)

In [ ]:
import scipy.stats
print( scipy.stats.ranksums( df_res['model acc'] , df_res['model acc_blur2x8']) )
print( scipy.stats.ranksums( df_res['model acc'] , df_res['model acc_blur5x8']) )
print( scipy.stats.ranksums( df_res['model acc'] , df_res['model acc_blur0x8']) )

In [ ]:
import scipy.stats
my_results = [scipy.stats.ranksums( df_res['model acc'] , df_res['model acc_blur2x8']),
              scipy.stats.ranksums( df_res['model acc'] , df_res['model acc_blur5x8']),
              scipy.stats.ranksums( df_res['model acc'] , df_res['model acc_blur0x8'])]
print(my_results)

my_blur = ['2x8', '5x8', '0x8']
for i in range(len(my_blur)):
    blur = my_blur[i]
    stat, pval = my_results[i]
    if (pval < 0.05/len(my_blur)):
        print('there is a significant difference between origional model with no blur and blur ', blur)
        print('\twith p-value ', pval)

so we see a drop in model accuracy with increasing blur. But aside from max blur (0x8), the model accuracy fall off isn't bad.  
Now to next part of the question- is this equal across the phylogenic groups?

In [ ]:
#use the initial result, blur columns to anaylze effect of blur
df_all['delta_2x8'] = df_all['model acc'] - df_all['model acc_blur2x8']
df_all['delta_5x8'] = df_all['model acc'] - df_all['model acc_blur5x8']
df_all['delta_0x8'] = df_all['model acc'] - df_all['model acc_blur0x8']

In [ ]:
#check Clade2
pops = df_all['Clade2'].unique().tolist()
blurs = ['delta_2x8', 'delta_5x8', 'delta_0x8',]

my_results= []

for blr in blurs:
    print('\n', blr)
    for pop1 in pops:
        for pop2 in pops:
            
            #print( pop1, pop2, scipy.stats.ranksums( df_all[df_all['Clade2'] == pop1 ][blr] ,
            #                            df_all[df_all['Clade2'] == pop2 ][blr] ) )
            
            stat, pval = scipy.stats.ranksums( df_all[df_all['Clade2'] == pop1 ][blr] ,
                                               df_all[df_all['Clade2'] == pop2 ][blr] )
            
            if pval < 0.05/len(pops):
                statement = 'Significant difference between'+ pop1+ pop2+ 'with'+ scipy.stats.ranksums( df_all[df_all['Clade2'] == pop1 ][blr] ,
                                        df_all[df_all['Clade2'] == pop2 ][blr] ) 
                my_results.push(statement)
                print(statement)

In [ ]:
if (len(my_results) ==0):
    print('There is not a significant difference at Clade 2 level in model performance for any blur level')
else:
    print('There is a significant difference at Clade 2 level in model performance for any blur level, between')
    print(my_results)

In [ ]:
df_now = df_all[['Clade2', 'Clade 3']].copy().groupby(['Clade2', 'Clade 3']).count().reset_index()
ps1 = df_now['Clade2'].to_list()
ps2 = df_now['Clade 3'].to_list()
print(df_now)

my_record = []

for blr in blurs:
    print('\n', blr) 
    for i in range(len(ps1)):
        p1c1 = ps1[i]
        p1c2 = ps2[i]
        for j in range(len(ps1)):
            p2c1 = ps1[j]
            p2c2 = ps2[j]
            if (len(df_all[(df_all['Clade2'] == p1c1) & (df_all['Clade 3'] == p2c2)][blr])>0 | 
                len(df_all[(df_all['Clade2'] == p2c1) & (df_all['Clade 3'] == p2c2)][blr])>0):
                
                stat, pval = scipy.stats.ranksums( df_all[(df_all['Clade2'] == p1c1) & (df_all['Clade 3'] == p2c2)][blr] ,
                                        df_all[(df_all['Clade2'] == p2c1) & (df_all['Clade 3'] == p2c2)][blr] )

                if (pval < (0.05/len(ps1))):
                    print('(', p1c1, p2c2,'), (', p2c1, p2c2 ,')', 'RanksumsResult(statistic=', stat,', ','pvalue=', pval,')')
                    statement = '(' + p1c1, p2c2 + '), (' + p2c1, p2c2 +');'
                    my_record.append(statement)
            #print('(', p1c1, p2c2,'), (', p2c1, p2c2 ,')',  
            #      scipy.stats.ranksums( df_all[(df_all['Clade2'] == p1c1) & (df_all['Clade 3'] == p2c2)][blr] ,
            #                            df_all[(df_all['Clade2'] == p2c1) & (df_all['Clade 3'] == p2c2)][blr] ) )
            #print( 'here', len(df_all[(df_all['Clade2'] == p1c1) & (df_all['Clade 3'] == p1c2)][blr]) )
        

In [ ]:
if (len(my_record) == 0):
    #no pairs with signifigant difference were found, so the test would pass
    print('No signficant difference among clades was found')
else:
    print('A signficant difference among clades was found amoung the following clade paris:')
    print(my_record)